## AI agent

### Install required packages

In [ ]:
!pip install crewai==0.30.0 crewai_tools==0.2.3 langchain_community==0.0.29 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.7/160.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 91.0 MB/s eta

### Warning control

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Import libraries, APIs and LLM

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
# from utils import get_openai_api_key, get_serper_api_key

openai_api_key = ''
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
os.environ["SERPER_API_KEY"] = ''
os.environ["OPENAI_API_KEY"] = openai_api_key

## crewAI Tools

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, YoutubeVideoSearchTool,PDFSearchTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
youtube_tool = YoutubeVideoSearchTool()
pdf_tool = PDFSearchTool()


## Creating Agents

In [ ]:
data_analyst_agent = Agent(
    role="United Airlines Reservation Agent",
    goal="Refund a customer ({question}) according to the ({url})",
    backstory="Specializing in refunding, this agent "
              "uses the ({url}) United which can be searched online"
              "to provide crucial insights on refunding for customers",
    verbose=True,
    allow_delegation=False,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [ ]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Read and understand the United Refund Page. When user ask ({question}), provide them answer promptly according to provided url ({url})"
    ),
    expected_output=(
        "answer from user({question}) "
    ),
    agent=data_analyst_agent,
)



## Creating the Crew
+ The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
+ In the example below, it will run this hierarchically.
+ `manager_llm` lets you choose the "manager" LLM you want to use.



In [ ]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
refund_crew = Crew(
    agents=[data_analyst_agent],

    tasks=[data_analysis_task],

    manager_llm=ChatOpenAI(model="gpt-4o",
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew
+ Set the inputs for the execution of the crew.

In [ ]:
# Example data for kicking off the process
refund_crew_inputs = {
'question': 'I cancelled my flight 2 hours prior to departure time, can I still get a refund?',
'url' :'https://www.united.com/en/us/'
}


In [ ]:
### this execution will take some time to run
result = refund_crew.kickoff(inputs=refund_crew_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Read and understand the United Refund Page. When user ask (I cancelled my flight 2 hours prior to departure time, can I still get a refund?), provide them answer promptly according to provided url (https://www.united.com/en/us/)


> Entering new CrewAgentExecutor chain...
To address the current task, I need to have a complete understanding of the United Refund Page to provide an accurate and comprehensive answer. Since I cannot access the URL directly, I will delegate this task to the United Airlines Reservation Agent, who can then provide the necessary information. 

Action: Delegate work to co-worker
Action Input: {"coworker": "United Airlines Reservation Agent", "task": "Read and understand the United Refund Page", "context": "Please visit the United Airlines website at the URL https://www.united.com/en/us/, navigate to the refund policy section, and provide a detailed explanation of the refund policy, especially concerni

+ Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)

To determine the refund eligibility for a United Airlines flight canceled 2 hours prior to departure, you would need to review their refund policy in detail. From the initial search, it appears that United Airlines generally allows for a refund if you cancel your flight up to 24 hours before the scheduled departure time. However, for specific cancellations made 2 hours prior to departure, the eligibility for a refund might depend on the type of ticket purchased (refundable vs. non-refundable) and other specific terms and conditions outlined by the airline. For the most accurate and detailed information, it would be best to directly contact United Airlines customer service or review their official refund policy page, particularly focusing on sections that address cancellations close to the time of departure.